<a href="https://colab.research.google.com/github/amara929/amara929/blob/main/Streamlit_for_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit pyngrok plotly statsmodels seaborn

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error

st.set_page_config(page_title="Bitcoin Time Series Analysis", layout="wide")
st.title("📈 Bitcoin Time Series Forecasting & Investor Analysis")

uploaded_file = st.file_uploader("/content/Bitcoin.csv", type=["csv"])
if uploaded_file:
    df = pd.read_csv("/content/Bitcoin.csv")
    df.drop_duplicates(inplace=True)

    if 'Date' not in df.columns:
        st.warning("Missing 'Date' column in dataset.")
    else:
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        st.subheader("Preview of Dataset")
        st.write(df.head())

        # Visualize Price Distribution
        st.subheader("Low vs High Price Interactive Scatter Plot")
        fig = px.scatter(df, x='Low', y='High', color='Volume', title='Low vs High Price')
        st.plotly_chart(fig)

        st.subheader("Correlation Heatmap")
        fig2, ax = plt.subplots(figsize=(10, 6))
        sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f", ax=ax)
        st.pyplot(fig2)

        st.subheader("Boxplot of Volume")
        fig3, ax = plt.subplots()
        sns.boxplot(data=df[['Volume']], ax=ax)
        st.pyplot(fig3)

        st.subheader("Pairplot (sampled data)")
        sampled_df = df.sample(min(200, len(df)))  # reduce for speed
        fig4 = sns.pairplot(sampled_df)
        st.pyplot(fig4)

        st.subheader("Seasonal Decomposition (High Prices)")
        try:
            decomposition = seasonal_decompose(df['High'], model='multiplicative', period=12)
            fig5, axs = plt.subplots(4, 1, figsize=(10, 8))
            decomposition.observed.plot(ax=axs[0], title='Observed')
            decomposition.trend.plot(ax=axs[1], title='Trend')
            decomposition.seasonal.plot(ax=axs[2], title='Seasonal')
            decomposition.resid.plot(ax=axs[3], title='Residual')
            plt.tight_layout()
            st.pyplot(fig5)
        except Exception as e:
            st.warning(f"Seasonal decomposition failed: {e}")

        st.subheader("Train SARIMA Model")
        train_size = int(len(df) * 0.8)
        train, test = df.iloc[:train_size], df.iloc[train_size:]

        train_data = train[['High']]
        test_data = test[['High']]

        try:
            model = SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
            model_fit = model.fit(disp=False)
            predictions = model_fit.predict(start=len(train), end=len(df)-1, dynamic=False)

            mae = mean_absolute_error(test['High'], predictions)
            mse = mean_squared_error(test['High'], predictions)
            rmse = np.sqrt(mse)

            st.success(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}")

            fig6, ax = plt.subplots(figsize=(12, 5))
            ax.plot(train.index, train['High'], label='Train')
            ax.plot(test.index, test['High'], label='Test', color='orange')
            ax.plot(test.index, predictions, label='Predicted', color='red')
            ax.legend()
            st.pyplot(fig6)

            steps = st.slider("Select future steps to forecast", 1, 100, 30)
            forecast = model_fit.forecast(steps=steps)
            st.line_chart(forecast)

        except Exception as e:
            st.error(f"Model failed: {e}")

        st.subheader("Hypothesis Testing")
        if 'Adj Close' in df.columns and 'Volume' in df.columns:
            try:
                df['Volume_Level'] = pd.qcut(df['Volume'], 2, labels=['Low', 'High'])
                t_stat, p_value = stats.ttest_ind(
                    df[df['Volume_Level'] == 'Low']['Adj Close'],
                    df[df['Volume_Level'] == 'High']['Adj Close']
                )
                st.write(f"Two-sample t-test p-value: {p_value:.4f}")
                if p_value < 0.05:
                    st.success("Statistically significant difference found.")
                else:
                    st.info("No significant difference found.")
            except Exception as e:
                st.warning(f"T-test failed: {e}")
        else:
            st.info("Missing 'Adj Close' or 'Volume' column for hypothesis testing.")

else:
    st.info("/content/Bitcoin.csv")

Overwriting app.py


In [ ]:
# Install required dependencies
!pip install streamlit pyngrok

# Import required libraries
from pyngrok import ngrok
import os

# Set your ngrok auth token (Make sure to replace this with your actual token)
from pyngrok import conf
conf.get_default().auth_token = "2vZiU5W9H7a3KEwplhCvW7nyAYt_2R4zc4XRfoCyed2y9yeYZ"

# Kill any previous ngrok tunnels
ngrok.kill()

# Run Streamlit app in the background
# Use os.system or subprocess to run the app
os.system("nohup streamlit run /content/app.py --server.port 8501 &")

# Expose Streamlit app through ngrok on port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")
import subprocess

subprocess.Popen(['nohup', 'streamlit', 'run', 'app.py', '--server.port', '5000', '&'])

Streamlit app is live at: NgrokTunnel: "https://09d1-34-16-235-226.ngrok-free.app" -> "http://localhost:8501"


<Popen: returncode: None args: ['nohup', 'streamlit', 'run', 'app.py', '--se...>